In [6]:
import csv
import pandas as pd

data = pd.read_csv('fbposts1.csv')

In [7]:
print(len(data))
data.head()
group_data = data.groupby('imdb_id')
# print(len(group_data))

48269


In [11]:
duration_dict = dict()
df = pd.read_csv('out_time.csv')
df_group = df.groupby('imdb_id')
for movie in df_group:
    movie_id = movie[0]
    duration_dict[movie_id] = movie[1]['duration'].max()
print(len(duration_dict))

203


In [12]:
count = 0
type_list = ['video', 'status', 'offer', 'swf', 'photo', 'link', 'question']
status_list = ['nan',
 'wall_post',
 'created_note',
 'created_event',
 'added_video',
 'mobile_status_update',
 'app_created_story',
 'added_photos',
 'shared_story']

slice_list = ['t1', 't2','t3']

def write_dict(slice_dict, key, t, s):
    slice_dict[key]['total'] += 1
    slice_dict[key][t] += 1
    slice_dict[key][s] += 1
moive_slice_dict = dict()
for movie in group_data:
    movie_id = movie[0]
    release_time = movie[1]['release_time'].min()
    first_pub_time = movie[1]['pub_time'].min()
    last_pub_time = movie[1]['pub_time'].max()
    duration = duration_dict[movie_id] + 1 # 加不加一就改这里
    out_time = release_time + duration - 1

    pub_before_release = release_time - first_pub_time
    pub_after_release = last_pub_time - out_time
    # print(movie_id, release_time, duration, out_time, first_pub_time, last_pub_time, pub_before_release, pub_after_release)


    # init slice_dict
    slice_dict = dict()
    for s in slice_list:
        slice_dict[s] = dict()
        slice_dict[s]['total'] = 0
        for t in type_list:
            slice_dict[s][t] = 0
        for t in status_list:
            slice_dict[s][t] = 0
    # print(movie[1])
    for i, com in movie[1].iterrows():
        # print(com['pub_time'])
        # print(com)
        pub_time = com['pub_time']
        t = com['type']
        s = str(com['statusType'])
        if pub_time < release_time:
            write_dict(slice_dict, 't1',t,s)
        elif pub_time > out_time:
            write_dict(slice_dict, 't3',t,s)
        else:
            write_dict(slice_dict, 't2',t,s)
        # print(com)
    # print(slice_dict)
    for key in slice_list:
        if key in ['t1']:
            slice_dict[key]['average'] = slice_dict[key]['total']/(release_time - first_pub_time)
            slice_dict[key]['time'] = release_time - first_pub_time
        elif key in ['t3']:
            slice_dict[key]['average'] = slice_dict[key]['total']/(last_pub_time - out_time) # 加不加一就改这里
            slice_dict[key]['time'] = last_pub_time - out_time
        else:
            slice_dict[key]['average'] = slice_dict[key]['total']/duration
            slice_dict[key]['time'] = duration
    count += 1
    slice_dict['duration'] = duration
    slice_dict['first_pub_before_release'] = pub_before_release
    slice_dict['last_pub_after_out'] = pub_after_release
    slice_dict['release_time'] = release_time
    slice_dict['out_time'] = out_time
    moive_slice_dict[movie_id] = slice_dict
    # break
print(count)

182


In [13]:
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
book = Workbook()
sheet = book.active
sheet['A1'] = 'imdb_id'
# sheet['C1'] = 'title'
sheet['B1'] = 'release_time'
sheet['C1'] = 'duration'
sheet['D1'] = 'out_time'
sheet['E1'] = 'first_pub_before_release'
sheet['F1'] = 'last_pub_after_out'
base_list = ['release_time','duration','out_time','first_pub_before_release','last_pub_after_out']
all_list = type_list + status_list + ['average']
# print(all_list)

k = 2

for b in base_list:
    e = get_column_letter(k)
    sheet[f'{e}{1}'] = b
    k += 1

for t in all_list:
    for s in slice_list:
        e = get_column_letter(k)
        sheet[f'{e}1'] = f'{t}_{s}'
        k += 1
        # print(f'{t}_{s}')
# for i, e in enumerate(['K','L','M','N','O','P','Q','R','S']):
#     sheet[f'{e}1'] = status_list[i]
# sheet['K1'] = 'total'
# sheet['L1'] = 'time'
# sheet['M1'] = 'average'
row = 2
for id, movie in moive_slice_dict.items():
    sheet[f'A{row}'] = id
    k = 2
    for b in base_list:
        e = get_column_letter(k)
        sheet[f'{e}{row}'] = movie[b]
        k += 1

    for t in all_list:
        for s in slice_list:
            e = get_column_letter(k)
            sheet[f'{e}{row}'] = movie[s][t]
            k += 1
    row += 1 
    # for i, t in enumerate(type_list):
    #     sheet[f'C{row}'] = t
    #     for j, e in enumerate(['D','E','F','G','H','I','J']):
    #         sheet[f'{e}{row}'] = movie[slice_list[j]][t]
    #     row += 1

    # for i, s in enumerate(status_list):
    #     sheet[f'C{row}'] = s
    #     for j, e in enumerate(['D','E','F','G','H','I','J']):
    #         sheet[f'{e}{row}'] = movie[slice_list[j]][s]
    #     row += 1

    # for i, s in enumerate(['total','time','average']):
    #     sheet[f'C{row}'] = s
    #     for j, e in enumerate(['D','E','F','G','H','I','J']):
    #         sheet[f'{e}{row}'] = movie[slice_list[j]][s]
    #     row += 1
book.save('movie_t123.csv')